<a href="https://colab.research.google.com/github/tarvt/DeleSmell_SA_G5/blob/main/dataset_class/flink-benchmarks-master/filink_semantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 KB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53924 sha256=fb5a6fb2c387e2174df9db741836d004586f27077b13b7eceeac3f93082f5e3a
  Stored in directory: /root/.cache/pip/wheels/78/ae/2e/5ce789557ca59281a463dabe9d6522e429be240322148e5379
Successfully built docx


In [ ]:
!pip install exceptions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement exceptions (from versions: none)
ERROR: No matching distribution found for exceptions


In [ ]:
import pandas as pd
import os
import numpy as np
import re
from string import punctuation
import pandas as pd
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
save_file_csv = 'flinkall.csv'
file_csv =  'flink.csv'
file_dir = '/content/drive/MyDrive/delesmell/dataset_class/flink-benchmarks-master' #Newly stored class address


In [ ]:
fileList = []
def file_name(file_dir):
    for root, dirs, files in os.walk(file_dir):
#       print(root) #current directory path
#       print(files) #All non-directory subfiles in the current path
         for file in files:
            if file[-4:] == '.txt':
                fileList.append(os.path.join(root, file))

In [ ]:
file_name(file_dir)

In [ ]:
fileList

['/content/drive/MyDrive/delesmell/dataset_class/flink-benchmarks-master/DeployingDownstreamTasksInBatchJobBenchmarkExecutor.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/flink-benchmarks-master/KeyByBenchmarks.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/flink-benchmarks-master/IntegerLongSource.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/flink-benchmarks-master/TestUtils.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/flink-benchmarks-master/QueuingLongSource.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/flink-benchmarks-master/PojoSerializationBenchmark.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/flink-benchmarks-master/RecordSource.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/flink-benchmarks-master/StreamNetworkLatencyBenchmarkExecutor.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/flink-benchmarks-master/LongNewSource.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/flink-benchmarks-master/SumReduceI

In [ ]:
# stopwords
stopwords = ['abstract', 'java','assert', 'boolean', 'break', 'byte', 'case', 'catch', 'class','char', 'continue', 'default', 'do', 'double', 'else', 'enum', 'extends', 'final', 'finally', 'float', 'for', 'if', 'implements', 'import', 'int', 'interface', 'long', 'native', 'new', 'package','private','public', 'protected',  'return', 'short', 'static', 'strictfp', 'super', 'switch', 'synchronized', 'this', 'throw', 'transient', 'try', 'void', 'volatile', 'while']
dicts = {'\n': '','!': '', '"': '','：':'', '#': '', '$': '', '%': '', '&': '', "'": '', '(': '', ')': '', '*': '', '+': '', ',': '', '-': '', '.': '', '/': '', ':': '', ';': '', '<': '', '=': '', '>': '', '?': '', '@': '', '[': '', '\\': '', ']': '', '^': '', '_': '', '`': '', '{': '', '|': '', '}': '', '~': ''}
# stopwords = {'abstract':'', 'java':'','assert':'', 'boolean':'', 'break':'', 'byte':'', 'case':'', 'catch':'', 'class':'','char':'', 'continue':'', 'default':'', 'do':'', 'double':'', 'else':'', 'enum':'', 'extends':'', 'final':'', 'finally':'', 'float':'', 'for':'', 'if':'', 'implements':'', 'import':'', 'int':'', 'interface':'', 'long':'', 'native':'', 'new':'', 'package':'','private':'','public':'', 'protected':'',  'return':'', 'short':'', 'static':'', 'strictfp':'', 'super':'', 'switch':'', 'synchronized':'', 'this':'', 'throw':'', 'transient':'', 'try':'','void':'', 'volatile':'', 'while':''}

In [ ]:
def lm_find_unchinese(line):
    pattern = re.compile(r'[\u4e00-\u9fa5]')
    unchinese = re.sub(pattern,"",line)
    unchinese = re.sub('[{}]'.format(punctuation),"",unchinese)
    #print("unchinese:",unchinese)
    return unchinese

In [ ]:
def del_stopwords(line):

    punc_table = str.maketrans(dicts)
    new_line = line.translate(punc_table)

    return new_line


In [ ]:
index = []
index_all = []
class_name = []
Brain_Class = []
for i, f in enumerate(fileList):
    with open(f,encoding='gb18030',errors='ignore') as file:
        list = []
        list2 = []
        is_brain = 0
        filename = os.path.basename(file.name)
        name = filename[:-4]
        class_name.append(name)
        list.append(name)
        list2.append(name)
        
        for line in file.readlines():
            line = lm_find_unchinese(line)
            line = del_stopwords(line)
            values = line.split(' ')
            coefs = values[1:-1]
            coefs = [i for i in coefs if i != '']
            keyword_brain = 'hongshuai'            

            if  keyword_brain in line:
                is_brain = 1
            if len(coefs) != 0:
                if coefs != ' ':
                    if coefs != '\n':
                        for i in coefs:
                            list.append(i)

        index.append(list)
        
        if is_brain == 0:
            list2.append(False)
        else :
            list2.append(True)
            print('T')
        
#         print (list2)
        
        Brain_Class.append(list2)
            


In [ ]:
Brain_Class = np.array(Brain_Class)
Brain_Class

array([['DeployingDownstreamTasksInBatchJobBenchmarkExecutor', 'False'],
       ['KeyByBenchmarks', 'False'],
       ['IntegerLongSource', 'False'],
       ['TestUtils', 'False'],
       ['QueuingLongSource', 'False'],
       ['PojoSerializationBenchmark', 'False'],
       ['RecordSource', 'False'],
       ['StreamNetworkLatencyBenchmarkExecutor', 'False'],
       ['LongNewSource', 'False'],
       ['SumReduceIntLong', 'False'],
       ['DataSkewStreamNetworkThroughputBenchmarkExecutor', 'False'],
       ['StringSerializationBenchmark', 'False'],
       ['ValidatingCounter', 'False'],
       ['DeployingTasksInStreamingJobBenchmarkExecutor', 'False'],
       ['MultipleInputBenchmark', 'False'],
       ['ListStateBenchmark', 'False'],
       ['MultiplyIntLongByTwo', 'False'],
       ['SortingBoundedInputBenchmarks', 'False'],
       ['MyPojo', 'False'],
       ['MemoryStateBackendBenchmark', 'False'],
       ['IntLongApplications', 'False'],
       ['SchedulingAndDeployingBenchmarkExecut

In [ ]:
len(index)

61

In [ ]:
def additem(list):
    if len(list)<3:
        q = len(list)-1
        str = '_1'
        str2 = '_2'
        item = ''.join([list[q], str])
        item_2 = ''.join([list[q], str2])
        # item = list[q]
        list.append(item)
        list.append(item_2)
        # print(list)
    return list

for item in index_all:
    # print(item)
    item = additem(item)
    # print(item)

In [ ]:
news_df = np.array(index,dtype=object)
print(news_df)

[list(['DeployingDownstreamTasksInBatchJobBenchmarkExecutor', 'BenchmarkModeModeSingleShotTime', 'public', 'void', 'deployDownstreamTasks', 'throws', 'Exception', 'BenchmarkModeModeSingleShotTime', 'public', 'void', 'deployDownstreamTasks', 'throws', 'Exception'])
 list(['KeyByBenchmarks', 'OperationsPerInvocationvalueKeyByBenchmarksARRAYRECORDSPERINVOCATION', 'public', 'void', 'arrayKeyByFlinkEnvironmentContext', 'context', 'throws', 'Exception', 'StreamExecutionEnvironment', 'envaddSourcenew', 'IncreasingArraySourceARRAYRECORDSPERINVOCATION10keyBy0addSinknew', 'OperationsPerInvocationvalueKeyByBenchmarksTUPLERECORDSPERINVOCATION', 'public', 'void', 'tupleKeyByFlinkEnvironmentContext', 'context', 'throws', 'Exception', 'StreamExecutionEnvironment', 'envaddSourcenew', 'IncreasingTupleSourceTUPLERECORDSPERINVOCATION10keyBy0addSinknew', 'OperationsPerInvocationvalueKeyByBenchmarksARRAYRECORDSPERINVOCATION', 'public', 'void', 'arrayKeyByFlinkEnvironmentContext', 'context', 'throws', 'Exce

In [ ]:

def f_lsa(news_df, n):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(news_df)
    n_cap = 1
    svd_model = TruncatedSVD(n_components=n_cap)  
    X2 = svd_model.fit_transform(X)  

    terms = vectorizer.get_feature_names()
    # print("--------话题及关键词---------")
    for i, comp in enumerate(svd_model.components_):
        terms_comp = zip(terms, comp)
        sorted_terms = sorted(terms_comp, key=lambda x: x[1], reverse=True)[:7]
        # print("Topic " + str(i) + ": ")
        for t in sorted_terms:
            print(t[0])
    print(" ")

    D = svd_model.components_
    T = X2
    S = np.diagflat(svd_model.singular_values_)

    X = np.dot(T, S)
    X = np.dot(X, D)
    c = np.dot(X, X.T)

    acsm = None
    sum = 0
    for i in c:
        for j in i:
            if j < 0:
                j = 0
            sum += j
        acsm = sum / n / n
    # print("-------------acsm----------------------")
    # print(acsm)

    return acsm

In [ ]:
q = 0
acsm= []
for i in index:
    lists = []
    print("*************",i,"****************")
    # print("------------------------------------------------------------------")
    for j in i:
        lists.append(j)
    # lists = deleteDuplicatedElementFromList(lists)
    if len(lists) > 1:
        acsm.append(f_lsa(lists,len(i)))
    else :
        acsm.append(1)
#

# print(type(class_name))
# print(type(acsm))

************* ['DeployingDownstreamTasksInBatchJobBenchmarkExecutor', 'BenchmarkModeModeSingleShotTime', 'public', 'void', 'deployDownstreamTasks', 'throws', 'Exception', 'BenchmarkModeModeSingleShotTime', 'public', 'void', 'deployDownstreamTasks', 'throws', 'Exception'] ****************
benchmarkmodemodesingleshottime
deploydownstreamtasks
deployingdownstreamtasksinbatchjobbenchmarkexecutor
exception
public
throws
void
 
************* ['KeyByBenchmarks', 'OperationsPerInvocationvalueKeyByBenchmarksARRAYRECORDSPERINVOCATION', 'public', 'void', 'arrayKeyByFlinkEnvironmentContext', 'context', 'throws', 'Exception', 'StreamExecutionEnvironment', 'envaddSourcenew', 'IncreasingArraySourceARRAYRECORDSPERINVOCATION10keyBy0addSinknew', 'OperationsPerInvocationvalueKeyByBenchmarksTUPLERECORDSPERINVOCATION', 'public', 'void', 'tupleKeyByFlinkEnvironmentContext', 'context', 'throws', 'Exception', 'StreamExecutionEnvironment', 'envaddSourcenew', 'IncreasingTupleSourceTUPLERECORDSPERINVOCATION10key

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


integerlongsourcerecordofvar1keyvar1value
var2
return
throws
reduceintegerlongsourcerecord
integerlongsourcerecord
var1integerlongsourcerecord
 
************* ['DataSkewStreamNetworkThroughputBenchmarkExecutor', 'public', 'void', 'networkSkewedThroughputMultiEnvironment', 'context', 'throws', 'Exception', 'public', 'void', 'networkSkewedThroughputMultiEnvironment', 'context', 'throws', 'Exception'] ****************
exception
void
public
dataskewstreamnetworkthroughputbenchmarkexecutor
context
throws
networkskewedthroughputmultienvironment
 
************* ['StringSerializationBenchmark', 'String', 'generatechar', 'charsetint', 'char', 'buffernew', 'Random', 'randomnew', 'for', 'int', 'i0', 'i', 'length', 'i', 'return', 'new', 'public', 'String', 'stringRead', 'throws', 'IOException', 'return', 'public', 'byte', 'stringWrite', 'throws', 'IOException', 'ByteArrayOutputStream', 'buffernew', 'DataOutputView', 'outnew', 'return', 'String', 'generatechar', 'charsetint', 'char', 'buffernew', '

In [ ]:
c = []
for i in range(len(class_name)):
      for j in range(len(acsm)):
        if i==j:
            t=[class_name[i],acsm[j]]
            c.append(t)

# c = list(zip(class_name,acsm))
# print(type(c))
c = np.array(c)
# print(c)
# print(c[0][0])

In [ ]:
from keras.models import Model
from tensorflow.keras.layers import Input
from keras.layers import Dense

In [ ]:
def word2vec_train(index):
    list = []
    list.append(index[0])
    words = []

    additem(index)
    # for text in index:
    #     #     print(text)
    print(index)
    for word in index:
        if word not in words:
            words.append(word)

    # words = set(words)
    # print(words)

    word2int = {}
    for i,word in enumerate(words):
        word2int[word] = i
    print(word2int)



    WINDOW_SIZE=2
    data = []
    # for sentence in index:
    for idx,word in enumerate(index):
        for neighbor in index[max(idx - WINDOW_SIZE,0) : min(idx + WINDOW_SIZE,len(index))+ 1]:
            if neighbor != word:
                data.append([word, neighbor])
    #                 print(data)
    df = pd.DataFrame(data,columns=['input','label'])

    # print(df)

    ONE_HOT_DIM = len(words)
    #function to convert numbers to one hot vectors
    def to_one_hot_encoding(data_point_index):
        one_hot_encoding = np.zeros(ONE_HOT_DIM)
        one_hot_encoding[data_point_index] = 1
        return one_hot_encoding

    X = []
    Y = []

    for x, y in zip(df['input'],df['label']):
        X.append(to_one_hot_encoding(word2int[x]))
        Y.append(to_one_hot_encoding(word2int[y]))

    x_train = np.array(X)
    y_train = np.array(Y)


    #Defining the size of the embedding
    embed_size = 3
    xx = Input(shape=(x_train.shape[1],))
    yy = Dense(units=embed_size, activation='linear')(xx)
    yy =Dense(units=y_train.shape[1],activation='softmax')(yy)
    model = Model(inputs=xx, outputs=yy)
    model.compile(loss = 'categorical_crossentropy',optimizer = 'adam')
    #optimizing the network weights
    model.fit(x=x_train,y=y_train,
            batch_size=16,
            epochs=10
    )
    weights = model.get_weights()[0]
    # embedding = []
    # embedding.append[index[0]]

    for word in words:
        # list = []
        # list.append(word)
        q = weights[df.get(word)]

        arr_new = np.sum(q, axis=1)
        print('-----------------------')
        print(arr_new.shape)
        print('-----------------------')
        arr_new = arr_new.tolist()

        list.append(arr_new)

        break
        # list.append(q)
        # # print(weights[df.get(word)].shape)
        # print(word)
        # print(q)
        # print('---')
        # embedding.append(list)
    return list


In [ ]:
embedding = []
for item in index:
    embedding.append(word2vec_train(item))
print(embedding)


['DeployingDownstreamTasksInBatchJobBenchmarkExecutor', 'BenchmarkModeModeSingleShotTime', 'public', 'void', 'deployDownstreamTasks', 'throws', 'Exception', 'BenchmarkModeModeSingleShotTime', 'public', 'void', 'deployDownstreamTasks', 'throws', 'Exception']
{'DeployingDownstreamTasksInBatchJobBenchmarkExecutor': 0, 'BenchmarkModeModeSingleShotTime': 1, 'public': 2, 'void': 3, 'deployDownstreamTasks': 4, 'throws': 5, 'Exception': 6}
Epoch 1/10
3/3 [==============================] - 0s 5ms/step - loss: 2.0152
Epoch 2/10
3/3 [==============================] - 0s 4ms/step - loss: 2.0121
Epoch 3/10
3/3 [==============================] - 0s 4ms/step - loss: 2.0098
Epoch 4/10
3/3 [==============================] - 0s 8ms/step - loss: 2.0073
Epoch 5/10
3/3 [==============================] - 0s 5ms/step - loss: 2.0050
Epoch 6/10
3/3 [==============================] - 0s 7ms/step - loss: 2.0028
Epoch 7/10
3/3 [==============================] - 0s 5ms/step - loss: 2.0004
Epoch 8/10
3/3 [=========

In [ ]:
df =pd.read_csv("/content/drive/MyDrive/delesmell/dataset_class/flink-benchmarks-master/flink-benchmarks-master.csv")

FileNotFoundError: ignored

In [ ]:
x= df.iloc[:, 0]

In [ ]:
ngList = []

In [ ]:
for index in x:
#     print(index)
    q = 1
    for i in range(0,len(embedding)):
        st = str(embedding[i][0])
        st = st[1:]
#         print(st)
        if index == st:
            # embedding[i][1].reshape((1,3))
            # l = embedding[i][1].tolist()
            ngList.append(embedding[i][1])
            q = 0
            # break
#             print('*')
    if q==1 :
        ngList.append([[1, 1, 1]])



word2vec = []
list1 = []
list2 = []
list3 = []
for i in range(0,len(ngList)):
    word2vec.append(ngList[i][0])

for i in range(0,len(word2vec)):
    list1.append(word2vec[i][0])
for i in range(0,len(word2vec)):
    list2.append(word2vec[i][1])
for i in range(0,len(word2vec)):
    list3.append(word2vec[i][2])


In [ ]:
for i in range(0,len(Brain_Class)):
    if 'False' == Brain_Class[i][1]:
        q = 1
    else :
        print('*')

In [ ]:
len(Brain_Class[0][0])

In [ ]:
#添加新列
ngList=[] #准备一个列表，把新列的数据存入其中

for index in x:
#     print(index)
    q = 1
    for i in range(0,len(c)):
        st = str(c[i][0])
        st = st[1:]
#         print(st)
        if index == st:
            ngList.append(c[i][1])
            q = 0
#             print('*')
    if q==1 :
        ngList.append(1)
        

In [ ]:
brainList=[] #准备一个列表，把新列的数据存入其中

for index in x:
#     print(index)
    q = 1
    for i in range(0,len(c)):
        st = str(Brain_Class[i][0])
        st = st[1:]
        if index == st:
            brainList.append(Brain_Class[i][1])
            q = 0
    if q==1 :
        brainList.append('0')
        
        
print(len(brainList))
print(len(x))

In [ ]:
column=df.columns.tolist()

In [ ]:
df['acsm'] = ngList
df['word2vec'] = list1
df['word2vec2'] = list2
df['word2vec3'] = list3
df['is_brain'] = brainList
df.to_csv(save_file_csv,mode = 'a',index =False)

In [ ]:
df.head()